In [1]:
from pyspark.sql.types import *
from pyspark.sql.functions import *
from pyspark.sql.functions import col,when
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

In [2]:
# Import SparkConf class into program
from pyspark import SparkConf
# local[*]: run Spark in local mode with as many working processors as logical cores on your machine
# If we want Spark to run locally with 'k' worker threads, we can specify as "local[k]".
master = "local[*]"
# The `appName` field is a name to be shown on the Spark cluster UI page
app_name = "diabetes"
# Setup configuration parameters for Spark
spark_conf = SparkConf().setMaster(master).setAppName(app_name)
# Import SparkContext and SparkSession classes
from pyspark import SparkContext # Spark
from pyspark.sql import SparkSession # Spark SQL
# Method 1: Using SparkSession
spark = SparkSession.builder.config(conf=spark_conf).getOrCreate()
sc = spark.sparkContext
sc.setLogLevel('ERROR')

In [3]:
# load raw file
raw_df = spark.read.option("header", True).option("inferSchema", True) .csv("diabetes_nhanes.csv")

In [4]:
# file's schema
raw_df.printSchema()

root
 |-- _c0: integer (nullable = true)
 |-- SEQN: double (nullable = true)
 |-- DIQ010: integer (nullable = true)
 |-- DID040: string (nullable = true)
 |-- DIQ160: string (nullable = true)
 |-- DIQ170: string (nullable = true)
 |-- DIQ172: string (nullable = true)
 |-- DIQ175A: string (nullable = true)
 |-- DIQ175B: string (nullable = true)
 |-- DIQ175C: string (nullable = true)
 |-- DIQ175D: string (nullable = true)
 |-- DIQ175E: string (nullable = true)
 |-- DIQ175F: string (nullable = true)
 |-- DIQ175G: string (nullable = true)
 |-- DIQ175H: string (nullable = true)
 |-- DIQ175I: string (nullable = true)
 |-- DIQ175J: string (nullable = true)
 |-- DIQ175K: string (nullable = true)
 |-- DIQ175L: string (nullable = true)
 |-- DIQ175M: string (nullable = true)
 |-- DIQ175N: string (nullable = true)
 |-- DIQ175O: string (nullable = true)
 |-- DIQ175P: string (nullable = true)
 |-- DIQ175Q: string (nullable = true)
 |-- DIQ175R: string (nullable = true)
 |-- DIQ175S: string (nullable

In [5]:
raw_df = raw_df.drop('_c0')

In [6]:
raw_df.show(1,vertical=True)

-RECORD 0----------
 SEQN    | 93703.0 
 DIQ010  | 2       
 DID040  | NA      
 DIQ160  | NA      
 DIQ170  | NA      
 DIQ172  | NA      
 DIQ175A | NA      
 DIQ175B | NA      
 DIQ175C | NA      
 DIQ175D | NA      
 DIQ175E | NA      
 DIQ175F | NA      
 DIQ175G | NA      
 DIQ175H | NA      
 DIQ175I | NA      
 DIQ175J | NA      
 DIQ175K | NA      
 DIQ175L | NA      
 DIQ175M | NA      
 DIQ175N | NA      
 DIQ175O | NA      
 DIQ175P | NA      
 DIQ175Q | NA      
 DIQ175R | NA      
 DIQ175S | NA      
 DIQ175T | NA      
 DIQ175U | NA      
 DIQ175V | NA      
 DIQ175W | NA      
 DIQ175X | NA      
 DIQ180  | NA      
 DIQ050  | 2       
 DID060  | NA      
 DIQ060U | NA      
 DIQ070  | NA      
 DIQ230  | NA      
 DIQ240  | NA      
 DID250  | NA      
 DID260  | NA      
 DIQ260U | NA      
 DIQ275  | NA      
 DIQ280  | NA      
 DIQ291  | NA      
 DIQ300S | NA      
 DIQ300D | NA      
 DID310S | NA      
 DID310D | NA      
 DID320  | NA      
 DID330  | NA      


In [7]:
raw_df.count(), len(raw_df.columns)

(8897, 54)

In [8]:
need_to_remove = ['DIQ175L','DIQ175M','DIQ175N','DIQ175O','DIQ175P','DIQ175Q','DIQ175R','DIQ175S','DIQ175T','DIQ175U','DIQ175V','DIQ175W','DIQ175X','DIQ180','DIQ050','DID060','DIQ060U','DIQ070','DIQ230','DIQ240','DID250','DID260','DIQ260U','DIQ275','DIQ280','DIQ291','DIQ300S','DIQ300D','DID310S','DID310D','DID320','DID330','DID341','DID350','DIQ350U','DIQ360','DIQ080']
for column in need_to_remove:
    raw_df = raw_df.drop(column)

In [9]:
raw_df.show(1,vertical=True)

-RECORD 0----------
 SEQN    | 93703.0 
 DIQ010  | 2       
 DID040  | NA      
 DIQ160  | NA      
 DIQ170  | NA      
 DIQ172  | NA      
 DIQ175A | NA      
 DIQ175B | NA      
 DIQ175C | NA      
 DIQ175D | NA      
 DIQ175E | NA      
 DIQ175F | NA      
 DIQ175G | NA      
 DIQ175H | NA      
 DIQ175I | NA      
 DIQ175J | NA      
 DIQ175K | NA      
only showing top 1 row



In [10]:
# NA --> null
for c in raw_df.columns:
    raw_df = raw_df.withColumn(c, when(col(c)=='NA' ,None).otherwise(col(c)))

In [11]:
raw_df.show(1,vertical=True)

-RECORD 0----------
 SEQN    | 93703.0 
 DIQ010  | 2       
 DID040  | null    
 DIQ160  | null    
 DIQ170  | null    
 DIQ172  | null    
 DIQ175A | null    
 DIQ175B | null    
 DIQ175C | null    
 DIQ175D | null    
 DIQ175E | null    
 DIQ175F | null    
 DIQ175G | null    
 DIQ175H | null    
 DIQ175I | null    
 DIQ175J | null    
 DIQ175K | null    
only showing top 1 row



In [12]:
# raw_df_new=raw_df.dropna(thresh=5)

In [13]:
# raw_df_new.count()

In [14]:
columns = raw_df.columns
columns.pop(0)
columns

['DIQ010',
 'DID040',
 'DIQ160',
 'DIQ170',
 'DIQ172',
 'DIQ175A',
 'DIQ175B',
 'DIQ175C',
 'DIQ175D',
 'DIQ175E',
 'DIQ175F',
 'DIQ175G',
 'DIQ175H',
 'DIQ175I',
 'DIQ175J',
 'DIQ175K']

In [15]:
# for column in columns:
#     raw_df_new.select(column)\
#     .groupBy(raw_df_new[column]).agg({column:'count'}).orderBy(count(raw_df_new[column]).desc()).show()

In [16]:
# for column in columns:
#     raw_df_new.select(count(when(col(column).isNull(),column))/count('*')*100).show()

In [17]:
# remove rows contain too many null values 
def new_df_desc(df,input_thresh,columns):
    # drop rows contain more than ... null
    raw_df_new=df.dropna(thresh=input_thresh) # input_thresh: require how many non-NA values
    print(raw_df_new.count())
    for column in columns:
        # group counts for each feature
        raw_df_new.select(column)\
        .groupBy(raw_df_new[column]).agg({column:'count'}).orderBy(count(raw_df_new[column]).desc()).show()
        # null percentage for each feature
        raw_df_new.select(count(when(col(column).isNull(),column))/count('*')*100).show()
    return raw_df_new

In [18]:
raw_df_new = new_df_desc(raw_df,3,columns)

6768
+------+-------------+
|DIQ010|count(DIQ010)|
+------+-------------+
|     2|         5690|
|     1|          892|
|     3|          182|
|     9|            4|
+------+-------------+

+----------------------------------------------------------------------+
|((count(CASE WHEN (DIQ010 IS NULL) THEN DIQ010 END) / count(1)) * 100)|
+----------------------------------------------------------------------+
|                                                                   0.0|
+----------------------------------------------------------------------+

+------+-------------+
|DID040|count(DID040)|
+------+-------------+
|    50|           79|
|    40|           48|
|    55|           45|
|    45|           45|
|    60|           42|
|    58|           31|
|    59|           23|
|    62|           23|
|    30|           22|
|    48|           21|
|    54|           20|
|    52|           20|
|    65|           20|
|    47|           19|
|    53|           19|
|    35|           18|
|    46

+-------+--------------+
|DIQ175K|count(DIQ175K)|
+-------+--------------+
|     20|            29|
|   null|             0|
+-------+--------------+

+------------------------------------------------------------------------+
|((count(CASE WHEN (DIQ175K IS NULL) THEN DIQ175K END) / count(1)) * 100)|
+------------------------------------------------------------------------+
|                                                       99.57151300236407|
+------------------------------------------------------------------------+



### 'DIQ170'--> target variable

* split data by 'DIQ010': Doctor told you have diabetes

In [19]:
df_1 = raw_df_new[raw_df_new['DIQ010']==1] # has diabetes
df_2 = raw_df_new[raw_df_new['DIQ010']==2] # do not have
df_3 = raw_df_new[raw_df_new['DIQ010']==3] # borderline

In [20]:
for column in columns:
    # group counts for each feature
    df_1.select(column)\
        .groupBy(df_1[column]).agg({column:'count'}).orderBy(count(df_1[column]).desc()).show()
    # null percentage for each feature
    df_1.select(count(when(col(column).isNull(),column))/count('*')*100).show()

+------+-------------+
|DIQ010|count(DIQ010)|
+------+-------------+
|     1|          892|
+------+-------------+

+----------------------------------------------------------------------+
|((count(CASE WHEN (DIQ010 IS NULL) THEN DIQ010 END) / count(1)) * 100)|
+----------------------------------------------------------------------+
|                                                                   0.0|
+----------------------------------------------------------------------+

+------+-------------+
|DID040|count(DID040)|
+------+-------------+
|    50|           79|
|    40|           48|
|    45|           45|
|    55|           45|
|    60|           42|
|    58|           31|
|    59|           23|
|    62|           23|
|    30|           22|
|    48|           21|
|    54|           20|
|    65|           20|
|    52|           20|
|    53|           19|
|    47|           19|
|    46|           18|
|    35|           18|
|    38|           17|
|    42|           17|
|    57|    

for filtered df, people has diabetes would have no data for other features, check raw df

In [21]:
raw_df_1 = raw_df[raw_df['DIQ010']==1]

In [22]:
for column in columns:
    # group counts for each feature
    raw_df_1.select(column)\
        .groupBy(raw_df_1[column]).agg({column:'count'}).orderBy(count(raw_df_1[column]).desc()).show()
    # null percentage for each feature
    raw_df_1.select(count(when(col(column).isNull(),column))/count('*')*100).show()

+------+-------------+
|DIQ010|count(DIQ010)|
+------+-------------+
|     1|          893|
+------+-------------+

+----------------------------------------------------------------------+
|((count(CASE WHEN (DIQ010 IS NULL) THEN DIQ010 END) / count(1)) * 100)|
+----------------------------------------------------------------------+
|                                                                   0.0|
+----------------------------------------------------------------------+

+------+-------------+
|DID040|count(DID040)|
+------+-------------+
|    50|           79|
|    40|           48|
|    55|           45|
|    45|           45|
|    60|           42|
|    58|           31|
|    59|           23|
|    62|           23|
|    30|           22|
|    48|           21|
|    65|           20|
|    54|           20|
|    52|           20|
|    47|           19|
|    53|           19|
|    46|           18|
|    35|           18|
|    42|           17|
|    38|           17|
|    57|    

people has diabetes would have no data for other features --> 'DIQ010' could not be target variable

* split data by 'DIQ170': Ever told have health risk for diabetes

In [23]:
df_1_170 = raw_df_new[raw_df_new['DIQ170']==1] # yes
df_2_170 = raw_df_new[raw_df_new['DIQ170']==2] # no
df_3_170 = raw_df_new[raw_df_new['DIQ170']==9] # don't know

In [24]:
for column in columns:
    df_2_170.select(column)\
        .groupBy(df_1_170[column]).agg({column:'count'}).orderBy(count(df_2_170[column]).desc()).show()
    df_2_170.select(count(when(col(column).isNull(),column))/count('*')*100).show()

+------+-------------+
|DIQ010|count(DIQ010)|
+------+-------------+
|     2|         4807|
|     3|          103|
|     9|            1|
+------+-------------+

+----------------------------------------------------------------------+
|((count(CASE WHEN (DIQ010 IS NULL) THEN DIQ010 END) / count(1)) * 100)|
+----------------------------------------------------------------------+
|                                                                   0.0|
+----------------------------------------------------------------------+

+------+-------------+
|DID040|count(DID040)|
+------+-------------+
|  null|            0|
+------+-------------+

+----------------------------------------------------------------------+
|((count(CASE WHEN (DID040 IS NULL) THEN DID040 END) / count(1)) * 100)|
+----------------------------------------------------------------------+
|                                                                 100.0|
+---------------------------------------------------------------

using UDF to change data in 'DIQ170':

    1 --> has risk of getting diabetes

    0 --> do not have risk of getting diabetes

In [25]:
def check_170(input_string):
    if input_string == '1': # has risk of diabetes
        return 1
    else:
        return 0

In [125]:
def check_null(input_string):
    if input_string == None: # has risk of diabetes
        return None
    else:
        return str(1)

In [26]:
check_170_UDF = udf(lambda x:check_170(x))

In [119]:
check_null_UDF = udf(lambda x:check_null(x))

In [120]:
raw_df.show(1)

+-------+-------------+----------------+----------------+---------+---------+--------------+----------+----+---------+----+------------------+----------------------+-------------------+----------------+----------------+------------+
|   SEQN|have_diabetes|age_get_diabetes|have_prediabetes|have_risk|feel_risk|family_history|overweight| age|poor_diet|race|baby_weighed_over9|lack_physical_activity|high_blood_pressure|high_blood_sugar|high_cholesterol|hypoglycemic|
+-------+-------------+----------------+----------------+---------+---------+--------------+----------+----+---------+----+------------------+----------------------+-------------------+----------------+----------------+------------+
|93703.0|            2|            null|            null|     null|     null|          null|      null|null|     null|null|              null|                  null|               null|            null|            null|        null|
+-------+-------------+----------------+----------------+---------+-

In [ ]:
# replace values in 'DIQ170'
final_df = raw_df.withColumn('DIQ170',check_170_UDF(col('DIQ170')).cast('int'))

In [28]:
raw_df_new = new_df_desc(final_df,8,columns)

545
+------+-------------+
|DIQ010|count(DIQ010)|
+------+-------------+
|     2|          509|
|     3|           34|
|     9|            2|
+------+-------------+

+----------------------------------------------------------------------+
|((count(CASE WHEN (DIQ010 IS NULL) THEN DIQ010 END) / count(1)) * 100)|
+----------------------------------------------------------------------+
|                                                                   0.0|
+----------------------------------------------------------------------+

+------+-------------+
|DID040|count(DID040)|
+------+-------------+
|  null|            0|
+------+-------------+

+----------------------------------------------------------------------+
|((count(CASE WHEN (DID040 IS NULL) THEN DID040 END) / count(1)) * 100)|
+----------------------------------------------------------------------+
|                                                                 100.0|
+-----------------------------------------------------------

In [29]:
raw_pd_df = raw_df_new.toPandas()

In [30]:
for column in columns:
    print(column)
    print(raw_pd_df.groupby(by=column)['DIQ170'].agg(['count','mean']).reset_index()\
             .rename(columns={'count':column+'Count', 'mean':'DIQ170 Ratio'})\
             .sort_values(by=[column+'Count'], ascending=False))

DIQ010
   DIQ010  DIQ010Count  DIQ170 Ratio
0       2          509      0.522593
1       3           34      0.705882
2       9            2      0.500000
DID040
Empty DataFrame
Columns: [DID040, DID040Count, DIQ170 Ratio]
Index: []
DIQ160
  DIQ160  DIQ160Count  DIQ170 Ratio
1      2          321      0.492212
0      1          187      0.577540
2      9            3      0.333333
DIQ170
   DIQ170  DIQ170Count  DIQ170 Ratio
1       1          291           1.0
0       0          254           0.0
DIQ172
  DIQ172  DIQ172Count  DIQ170 Ratio
0      1          545      0.533945
DIQ175A
  DIQ175A  DIQ175ACount  DIQ170 Ratio
0      10           432      0.638889
DIQ175B
  DIQ175B  DIQ175BCount  DIQ170 Ratio
0      11           425      0.531765
DIQ175C
  DIQ175C  DIQ175CCount  DIQ170 Ratio
0      12           155      0.496774
DIQ175D
  DIQ175D  DIQ175DCount  DIQ170 Ratio
0      13           328           0.5
DIQ175E
  DIQ175E  DIQ175ECount  DIQ170 Ratio
0      14           129       0.55814

In [126]:
def df_cleaning(df_name):
    # load raw file
    df = spark.read.option('header', True).option('inferSchema', True) .csv(df_name)
    # drop useless feature
    need_to_remove = ['_c0','DIQ175C','DIQ175E','DIQ175L','DIQ175M','DIQ175N','DIQ175O','DIQ175P','DIQ175Q','DIQ175R','DIQ175S','DIQ175T','DIQ175U','DIQ175V','DIQ175W','DIQ175X','DIQ180','DIQ050','DID060','DIQ060U','DIQ070','DIQ230','DIQ240','DID250','DID260','DIQ260U','DIQ275','DIQ280','DIQ291','DIQ300S','DIQ300D','DID310S','DID310D','DID320','DID330','DID341','DID350','DIQ350U','DIQ360','DIQ080']
    for column in need_to_remove:
        df = df.drop(column)
    # NA --> null
    for c in df.columns:
        df = df.withColumn(c, when(col(c)=='NA' ,None).otherwise(col(c)))
    # target feature's value change --> [0,1]
    df = df.withColumn('DIQ170',check_170_UDF(col('DIQ170')).cast('int'))
    # rename columns
    df = df.withColumnRenamed('DIQ010', 'have_diabetes')
    df = df.withColumnRenamed('DID040', 'age_get_diabetes')
    df = df.withColumnRenamed('DIQ160', 'have_prediabetes')
    df = df.withColumnRenamed('DIQ170', 'have_risk')
    df = df.withColumnRenamed('DIQ172', 'feel_risk')
    df = df.withColumnRenamed('DIQ175A', 'family_history')
    df = df.withColumnRenamed('DIQ175B', 'overweight')
    df = df.withColumnRenamed('DIQ175D', 'poor_diet')
    df = df.withColumnRenamed('DIQ175F', 'baby_weighed_over9')
    df = df.withColumnRenamed('DIQ175G', 'lack_physical_activity')
    df = df.withColumnRenamed('DIQ175H', 'high_blood_pressure')
    df = df.withColumnRenamed('DIQ175I', 'high_blood_sugar')
    df = df.withColumnRenamed('DIQ175J', 'high_cholesterol')
    df = df.withColumnRenamed('DIQ175K', 'hypoglycemic')
    for column in ['overweight','poor_diet','baby_weighed_over9','lack_physical_activity','high_blood_pressure','high_blood_sugar','high_cholesterol','hypoglycemic']:
        df = df.withColumn(column,check_null_UDF(col(column)))
    
    # drop rows contain more than ... null
    df=df.dropna(thresh=6) # input_thresh: require how many non-NA values
    
    return df

In [127]:
df_17_18 = df_cleaning('diabetes_nhanes_17_18.csv')
df_15_16 = df_cleaning('diabetes_nhanes_15_16.csv')
df_13_14 = df_cleaning('diabetes_nhanes_13_14.csv')
df_11_12 = df_cleaning('diabetes_nhanes_11_12.csv')

In [128]:
union_all_df = df_17_18.union(df_15_16).cache()
union_all_df = union_all_df.union(df_13_14).cache()
union_all_df = union_all_df.union(df_11_12).cache()

In [129]:
union_all_df.count()

38010

In [130]:
union_all_df_pd = union_all_df.toPandas()

In [134]:
filtered_union_all_df=union_all_df.dropna(thresh=6)

In [135]:
filtered_union_all_df.count()

6141

In [91]:
columns = filtered_union_all_df.columns
columns.pop(0)
columns

['have_diabetes',
 'age_get_diabetes',
 'have_prediabetes',
 'have_risk',
 'feel_risk',
 'family_history',
 'overweight',
 'poor_diet',
 'baby_weighed_over9',
 'lack_physical_activity',
 'high_blood_pressure',
 'high_blood_sugar',
 'high_cholesterol',
 'hypoglycemic']

In [131]:
for column in columns:
    print(union_all_df_pd.groupby(by=column)['have_risk'].agg(['count','mean']).reset_index()\
             .rename(columns={'count':'Count', 'mean':'Have Risk Ratio'})\
             .sort_values(by=['Count'], ascending=False))

  have_diabetes  Count  Have Risk Ratio
1             2  34066         0.095902
0             1   3223         0.000000
2             3    701         0.455064
4             9     18         0.388889
3             7      2         0.000000
   age_get_diabetes  Count  Have Risk Ratio
45               50    239              0.0
34               40    159              0.0
50               55    157              0.0
39               45    154              0.0
56               60    140              0.0
..              ...    ...              ...
77               79      4              0.0
22                3      4              0.0
0                 1      3              0.0
55                6      1              0.0
5                14      1              0.0

[82 rows x 3 columns]
  have_prediabetes  Count  Have Risk Ratio
1                2  22612         0.113082
0                1   1666         0.420168
2                9     35         0.485714
   have_risk  Count  Have Risk Ratio


In [136]:
filtered_union_all_df_pd = filtered_union_all_df.toPandas()

In [133]:
union_all_df_pd.to_csv('union_all_diabetes_with_null.csv',index=False)

In [137]:
filtered_union_all_df_pd.to_csv('filtered_union_all_diabetes_less_null.csv',index=False)

In [138]:
for column in columns:
    print(filtered_union_all_df_pd.groupby(by=column)['have_risk'].agg(['count','mean']).reset_index()\
             .rename(columns={'count':'Count', 'mean':'Have Risk Ratio'})\
             .sort_values(by=['Count'], ascending=False))
    filtered_union_all_df.select(column)\
        .groupBy(filtered_union_all_df[column]).agg({column:'count'}).orderBy(count(union_all_df[column]).desc()).show()
    # null percentage for each feature
    filtered_union_all_df.select(count(when(col(column).isNull(),column))/count('*')*100).show()

  have_diabetes  Count  Have Risk Ratio
0             2   5827         0.370002
1             3    307         0.596091
2             9      7         0.714286
+-------------+--------------------+
|have_diabetes|count(have_diabetes)|
+-------------+--------------------+
|            2|                5827|
|            3|                 307|
|            9|                   7|
+-------------+--------------------+

+------------------------------------------------------------------------------------+
|((count(CASE WHEN (have_diabetes IS NULL) THEN have_diabetes END) / count(1)) * 100)|
+------------------------------------------------------------------------------------+
|                                                                                 0.0|
+------------------------------------------------------------------------------------+

Empty DataFrame
Columns: [age_get_diabetes, Count, Have Risk Ratio]
Index: []
+----------------+-----------------------+
|age_get_diabetes|count

+----------------+-----------------------+
|high_blood_sugar|count(high_blood_sugar)|
+----------------+-----------------------+
|               1|                    275|
|            null|                      0|
+----------------+-----------------------+

+------------------------------------------------------------------------------------------+
|((count(CASE WHEN (high_blood_sugar IS NULL) THEN high_blood_sugar END) / count(1)) * 100)|
+------------------------------------------------------------------------------------------+
|                                                                         95.52190197036313|
+------------------------------------------------------------------------------------------+

  high_cholesterol  Count  Have Risk Ratio
0                1    527         0.474383
+----------------+-----------------------+
|high_cholesterol|count(high_cholesterol)|
+----------------+-----------------------+
|               1|                    527|
|            null